# Example: Define hydrological model regions

The first step when building a model is to define the model domain. For hydrological models this is typically a full **basin** or **sub-basin** which includes all upstream cells. For hydrodynamic models this can also be an **inter-basin** which does not necessary include all upstream cells. *HydroMT* has the [get_basin_geometry()](https://deltares.github.io/hydromt/latest/_generated/hydromt.workflows.basin_mask.get_basin_geometry.html) method to do just that. 

## Import packages

In [ ]:
import geopandas as gpd
import numpy as np
from shapely.geometry import box

In [ ]:
# for plotting
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import matplotlib.pyplot as plt

proj = ccrs.PlateCarree()  # plot projection

In [ ]:
# import hydromt
import hydromt
from hydromt.model.processes.basin_mask import get_basin_geometry

## Read data

Read data using the [DataCatalog](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.html). If no yml files with data sources are provided, by default data for the Piave basin is downloaded from the hydromt-artifacts to to `~/.hydromt_data/`. Links to the original data sources and data licenses can be found in the meta dict of each source. Here we use flow direction data from [MERIT Hydro](http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro). Note that the original `'MERIT Hydro'` data has been extended with a basin mask layer, which together with the basin index vector data makes faster basin delineation possible.

In [ ]:
# instantiate instance of Data Catalog
data_catalog = hydromt.DataCatalog(data_libs=["artifact_data=v1.0.0"])

In [ ]:
# read MERIT hydro flow direction data
print(data_catalog.get_source("merit_hydro"))
ds = data_catalog.get_rasterdataset("merit_hydro")

In [ ]:
# read MERIT hydro basin index vector data. This data contains bounding box geometries of all basins globally.
# Here we pass the GeoDataFrameAdapter instead of the GeoDataFrame itself
# a spatial subset of the data loaded within the get_basin_geometry method
basin_index = data_catalog.get_source("merit_hydro_index")
print(basin_index)

In [ ]:
# derive river geometry based on stream order >= 7 (for plotting only)
flwdir = hydromt.gis.flw.flwdir_from_da(ds["flwdir"], ftype="d8")
feats = flwdir.streams(mask=ds["strord"] >= 7)
gdf_riv = gpd.GeoDataFrame.from_features(feats)


def plot(extent, gdf_bas, gdf_out):
    plt.figure(figsize=(3, 5))
    ax = plt.subplot(projection=proj)
    ax.set_extent(extent, crs=proj)
    ax.add_image(cimgt.QuadtreeTiles(), 10)
    gdf_bas.boundary.plot(ax=ax, edgecolor="k", zorder=2)
    gdf_riv.plot(ax=ax, color="blue", alpha=0.7)
    gdf_out.plot(ax=ax, markersize=40, c="red", zorder=2)
    return ax

## Delineate basins

Several examples to delineate sub-, inter- or full basins are provided below together with the command line interface (CLI) syntax for the HydroMT [build](https://deltares.github.io/hydromt/latest/guides/user_guide/model_build.html) command. All CLI options are described in the [parse_region()](https://deltares.github.io/hydromt/latest/_generated/hydromt.workflows.basin_mask.parse_region.html) method.

Get the basin based on a point location `[x, y]`.

In [ ]:
# CLI syntax: -r {'basin': [x, y]}
# e.g.: -r {'basin': [12.6, 45.8]}

xy = [12.6, 45.8]
gdf_bas, _ = get_basin_geometry(
    ds,
    kind="basin",
    xy=xy,
    basin_index=basin_index,
)

In [ ]:
# plot results
gdf_xy = gpd.GeoDataFrame(geometry=gpd.points_from_xy(x=[xy[0]], y=[xy[1]]), crs=4326)
extent = np.array(gdf_bas.buffer(0.1).total_bounds)[[0, 2, 1, 3]]

plot(extent, gdf_bas, gdf_xy)

## Delineate sub-basins

We need an initial bounding box to delineate the sub-basin. This can be derived from the *merit_hydro_index* or user provided initial `bounds`. 
The latter might be faster if delineating a small sub-basin from a large basin. A warning is raised if not all contributing cells are included.

Get a subbasin based on its outlet location `[x,y]`, snapped to a stream defined by a `<variable>:<threshold>` pair, in this case a stream order larger or equal to 7. The `variable` should be present in the dataset `ds`. 

In [ ]:
# CLI syntax: -r {'subbasin': [x, y], <variable>: <threshold>, 'bounds': [xmin, ymin, xmax, ymax]}
# e.g.: -r {'subbasin': [12.6, 45.8], 'strord': 7, 'bounds': [12.1, 45.5, 12.9, 46.5]}

xy = [12.6, 45.8]
bounds = [12.1, 45.5, 12.9, 46.5]
gdf_bas, gdf_out = get_basin_geometry(
    ds,
    kind="subbasin",
    xy=xy,
    strord=7,
    bounds=bounds,
)

In [ ]:
# plot results
gdf_bounds = gpd.GeoDataFrame(geometry=[box(*bounds)], crs=4326)
extent = gdf_bounds.buffer(0.05).total_bounds[[0, 2, 1, 3]]

ax = plot(extent, gdf_bas, gdf_out)
gdf_bounds.boundary.plot(ax=ax, edgecolor="k", ls="--")

Get a sub-basin based on the location where the stream flows out of a `bbox`. Here the stream defined by a `<variable>:<threshold>` pair, in this case a stream order larger or equal to 8. The `variable` should be present in the dataset `ds`. 

In [ ]:
# CLI SYNTAX: -r {'subbasin': [xmin, ymin, xmax, ymax], <variable>: <threshold>}
# e.g.: -r {'subbasin': [12.50, 45.72, 12.7, 46], 'strord': 8}

bbox = [12.50, 45.72, 12.7, 46]
gdf_bas, gdf_out = get_basin_geometry(
    ds,
    kind="subbasin",
    bbox=bbox,
    strord=8,
    basin_index=basin_index,
)

In [ ]:
# plot results
gdf_bbox = gpd.GeoDataFrame(geometry=[box(*bbox)], crs=4326)
extent = gdf_bas.buffer(0.05).total_bounds[[0, 2, 1, 3]]

ax = plot(extent, gdf_bas, gdf_out)
gdf_bbox.boundary.plot(ax=ax, edgecolor="r", ls="-")

## Delineate interbasins

Get an inter-basin based on the streams within a `bbox`. The inter-basin is limited to the most downstream contiguous area within the bbox that drains to the stream.  Here the stream defined by a `<variable>:<threshold>` pair, in this case a stream order larger or equal to 8. The `variable` should be present in the dataset `ds`. The `buffer` is required to check wether streams flow in and out of the bbox. 

In [ ]:
# CLI SYNTAX -r {'interbasin': [xmin, ymin, xmax, ymax], <variable>: <threshold>}
# e.g.: -r {'interbasin': [12.50, 45.72, 12.7, 46], 'strord': 8}

bbox = [12.50, 45.72, 12.7, 46]
gdf_bas, gdf_out = get_basin_geometry(
    ds,
    kind="interbasin",
    bbox=bbox,
    strord=8,
    buffer=20,
)

In [ ]:
# plot results
gdf_bbox = gpd.GeoDataFrame(geometry=[box(*bbox)], crs=4326)
extent = gdf_bbox.buffer(0.02).total_bounds[[0, 2, 1, 3]]

ax = plot(extent, gdf_bas, gdf_out)
gdf_bbox.boundary.plot(ax=ax, edgecolor="r", ls="-", zorder=1)